## DuckDB Python API
- the same way of loading sakila data as via terminal


In [6]:
import duckdb
from pathlib import Path # without pathlib you need to delete everything

In [1]:
import pandas

ImportError: C extension: pandas.util not built. If you want to import pandas from the source directory, you may need to run 'python setup.py build_ext' to build the C extensions first.

### read data frame one by one

In [7]:
# create variable to store duckdb path
duckdb_path = "data/sakila.duckdb"
Path(duckdb_path).unlink(missing_ok=True) 

# connect to duckdb file with context manager,
# and ingest the data. 
# same as duckdb sakila.duckdb < sql/laod_sakila.sql
with duckdb.connect(duckdb_path) as conn, open("sql/load_sakila.sql") as ingest_scripts:
    conn.sql(ingest_scripts.read())
    # create pandas dataframes from the created duckdb connection
    description = conn.sql("DESC;").df()
    film = conn.sql("FROM film;").df()

# check the created dataframe
film.head(2)


InvalidInputException: Invalid Input Error: 'pandas' is required for this operation but it was not installed

In [ ]:
description.head(3)

,database,schema,name,column_names,column_types,temporary
0,sakila,main,actor,"[actor_id, first_name, last_name, last_update]","[DOUBLE, VARCHAR, VARCHAR, TIMESTAMP]",False
1,sakila,main,address,"[address_id, address, address2, district, city...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, BIGINT, VA...",False
2,sakila,main,category,"[category_id, name, last_update]","[BIGINT, VARCHAR, TIMESTAMP]",False


### read all tables into dataframes

In [ ]:
# create an empty dictionary to keep all dataframes
dfs = {}

with duckdb.connect(duckdb_path) as conn: 
    for name in description["name"]:
        dfs[name] = conn.sql(f"FROM {name};").df() # to store all dataframes from description by column 'name' into dictionary
        



In [ ]:
# check the resulting dictionary
dfs.keys()

dict_keys(['actor', 'address', 'category', 'city', 'country', 'customer', 'customer_list', 'film', 'film_actor', 'film_category', 'film_list', 'film_text', 'inventory', 'language', 'payment', 'rental', 'sales_by_film_category', 'sales_by_store', 'staff', 'staff_list', 'store'])

In [ ]:
# each value for each key in the dictionary is a panda dataframe
type(dfs["actor"])

pandas.core.frame.DataFrame

In [ ]:
# check the content of the df
dfs["actor"].head(3)

,actor_id,first_name,last_name,last_update
0,1.0,PENELOPE,GUINESS,2021-03-06 15:51:59
1,2.0,NICK,WAHLBERG,2021-03-06 15:51:59
2,3.0,ED,CHASE,2021-03-06 15:51:59


### register dfs to duckdb (tables) by joining dataframes

In [ ]:
# duckdb.register("actor", dfs["actor"]) -one by one


## join the register tables

In [ ]:
table_names = ["film", "film_actor", "film_category", "actor", "category"]

for table_name in table_names:
    duckdb.register(table_name, dfs[table_name])

duckdb.sql("DESC;").df()

NameError: name 'duckdb' is not defined

## EDA with pandas
- a light introduction of pandas

## Visualization with pandas dataframe